[長岡技術科学大学 自然言語研究室 やさしい日本語コーパス](http://www.jnlp.org/SNOW/T15)

[NTCIR-10 RITE2](http://www.cl.ecei.tohoku.ac.jp/rite2/doku.php?id=wiki:%E3%83%A1%E3%82%A4%E3%83%B3%E3%83%9A%E3%83%BC%E3%82%B8)

[京都大学 黒橋・河原研究室 Textual Entailment評価データ](http://nlp.ist.i.kyoto-u.ac.jp/index.php?Textual%20Entailment%20%E8%A9%95%E4%BE%A1%E3%83%87%E3%83%BC%E3%82%BF)

[日本語学習済みBERT](https://yoheikikuta.github.io/bert-japanese/)

以上のデータを学習用に使っています。
学習を回す際はダウンロードして適当にpathを設定してください。


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install text_vectorian

     |████████████████████████████████| 1.0MB 12.7MB/s 
  Created wheel for keras-bert: filename=keras_bert-0.80.0-cp36-none-any.whl size=37923 sha256=021e7f7e5dafbe7f6e550625822d99133cb426916eeaae20b1608c6593008fdf
  Stored in directory: /root/.cache/pip/wheels/63/dc/87/3260cb91f3aa32c0f85c5375429a30c8fd988bbb48f5ee21b0
  Created wheel for keras-transformer: filename=keras_transformer-0.31.0-cp36-none-any.whl size=13385 sha256=bc604d9a0f08f78484ebab7758485a567eb0a31626082e16829a83e96c2a0cbc
  Stored in directory: /root/.cache/pip/wheels/a3/c5/9a/5a5130240be614a7a6fa786765d7692ae97f82601e2161bb56
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7553 sha256=8537865cf986405ef5e2f98e4d4c06d26c209d6dbb9419c3cb31a85be3c5f67e
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.22.0-cp36-none-any.whl size=15371 sha256=fc34eafd97603cd

In [0]:
from text_vectorian import SpBertVectorian
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import  numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Activation,Dropout,Input,Reshape,Flatten,AveragePooling2D,GlobalAveragePooling2D
from keras.layers import Conv2D,MaxPooling2D,Concatenate,BatchNormalization
from keras.models import Model
from keras import Input
import datetime
import keras

In [0]:

def data_load():
    train_x = []
    train_y = []
    
    true_count = 0
    
    entail_kyoto = ET.parse("/content/drive/My Drive/Aizen/Datas/entail_evaluation_set_label_conv.xml") #京大コーパスの読み込み
    entail_kyoto_root = entail_kyoto.getroot()
    for child in entail_kyoto_root:
        temp = []
        if child.attrib["label"] == "Y":
            train_y.append(1)
            true_count += 1
        else:
            train_y.append(0)
        for gchild in child:
            temp.append(gchild.text)
        train_x.append(temp)
    RITE_names = ["dev_bc","dev_mc","testlabel_bc","testlabel_mc"]
    pos = ["F","B","Y"]
    neg = ["C","I","N"]
    for name in RITE_names: # RITEの読み込み。
        rite_file_path = "/content/drive/My Drive/Aizen/Datas/RITE2_JA_bc-mc-unittest_forOpenAccess/RITE2_JA_{}/RITE2_JA_{}.xml".format(name,name)
        rite_tree = ET.parse(rite_file_path)
        root = rite_tree.getroot()
        for child in root:
            temp = []
            if child.attrib["label"] in pos:
                train_y.append(1)
                true_count += 1
            else:
                train_y.append(0)
            for gchild in child:
                temp.append(gchild.text)
            train_x.append(temp)
    easy_jp = pd.read_csv("/content/drive/My Drive/Aizen/Datas/T15-2020.1.7.csv").values #やさしい日本語の読み込み
    for line in easy_jp:
        if line[1] != line[2]:
            train_y.append(1)
            train_x.append([line[1],line[2]])
            true_count += 1
            
    all_num = len(train_x)
    
    for i in range(2*true_count-all_num): #無理やり均衡化。ランダムに文を組み合わせて非含意データを増やす。
        left_raw = np.random.randint(all_num)
        left_col = np.random.randint(2)
        right_raw = np.random.randint(all_num)
        while left_raw == right_raw:
            right_raw = np.random.randint(all_num)
        right_col = np.random.randint(2)
        train_x.append([train_x[left_raw][left_col],train_x[right_raw][right_col]])
        train_y.append(0)
    
    
    
    return train_x,train_y

In [5]:
vectorizer_filename = f'/content/drive/My Drive/Aizen/bert-wiki-ja/model.ckpt-1400000'
tokenizer_filename = f'/content/drive/My Drive/Aizen/bert-wiki-ja/wiki-ja.model'
vectorian = SpBertVectorian(
    tokenizer_filename=tokenizer_filename,
    vectorizer_filename=vectorizer_filename
)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









In [0]:
all_x,all_y = data_load()

In [0]:
def make_network():
    x_in = Input((256,768,1))
    x = Conv2D(64,(3,3),padding="valid")(x_in)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64,(3,3),padding="valid")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(3,3),strides=2,padding="same")(x)

    x = Conv2D(64,(2,2),padding="valid")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128,(2,2),padding="valid")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(3,3),strides=2,padding="same")(x)

    x = GlobalAveragePooling2D()(x)

    x = Dense(2)(x)
    x = Activation("softmax")(x)
    model = Model(x_in,x)
    return model

In [0]:
def data_gen(data_X,data_y,vectorian,batchsize=32):
    d_cnt = 0
    x = []
    y = []
    while True:
        for i in range(len(data_X)):
            t_l = str(data_X[i][0])
            t_r = str(data_X[i][1])

            t_l = vectorian.fit(t_l).vectors
            t_r = vectorian.fit(t_r).vectors


            itp = range(t_l.shape[1])
            in_x = np.insert(t_r,itp,t_l[:,itp],axis=1) #ベクトルを互い違いに入れて一つの入力データにまとめる
            
            x.append(in_x)
            
            if data_y[i] == 1:
                y.append([0,1])
            else:
                y.append([1,0])
            
            d_cnt += 1
            
            if d_cnt == batchsize:
                inputs = np.array(x).astype(np.float32)
                inputs = inputs.reshape(batchsize,256,768,1)
                targets = np.array(y)
                x = []
                y = []
                d_cnt = 0
                yield inputs,targets
        

In [0]:
train_x,valid_x,train_y,valid_y = train_test_split(all_x,all_y,shuffle=True,random_state=1)

In [12]:
print(vectorian.fit("vectorianを慣らすよ").vectors) #これをやらないと何故かエラーが出る。

[[-0.11794432  1.2328457   0.7276289  ... -0.2198706  -0.08330172
   0.39365828]
 [ 0.44415194 -0.7618504  -0.36340111 ...  1.9161105   0.27549568
   0.01428878]
 [ 0.64637655 -0.7549831  -0.40080702 ...  0.99348027  0.69800144
  -0.32642797]
 ...
 [ 0.09190618 -0.8589261   0.04196239 ...  1.061703   -0.31088674
   0.7522747 ]
 [ 0.21613017 -0.45978865 -0.33493537 ...  0.7047594  -0.28041598
   0.65170586]
 [-0.5390734   0.34215537  0.34547573 ... -0.3640052   0.40427518
   0.4227317 ]]


In [15]:
model = make_network()
# model = keras.models.load_model("/content/drive/My Drive/Aizen/my_model"+str(1)+".h5", compile=False)
model.summary()
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=["categorical_accuracy"])



Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 256, 768, 1)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 254, 766, 64)      640       
_________________________________________________________________
batch_normalization_9 (Batch (None, 254, 766, 64)      256       
_________________________________________________________________
activation_11 (Activation)   (None, 254, 766, 64)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 252, 764, 64)      36928     
_________________________________________________________________
batch_normalization_10 (Batc (None, 252, 764, 64)      256       
_________________________________________________________________
activation_12 (Activation)   (None, 252, 764, 64)      0 

In [16]:
batchsize=24
epochs=1
save_ind = 2
#学習。1epoch1時間くらいかかるので、毎回モデルセーブ
for i in range(10):
  spe = len(train_x)//batchsize
  spev = len(valid_x)//batchsize
  train_gen = data_gen(train_x,train_y,vectorian,batchsize=batchsize)
  valid_gen = data_gen(valid_x,valid_y,vectorian,batchsize=batchsize)
  model.fit_generator(train_gen,epochs=epochs,steps_per_epoch=spe,validation_data=valid_gen,validation_steps=spev)
  model.save("/content/drive/My Drive/Aizen/my_model"+str(save_ind)+".h5")
  save_ind += 1

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1
1955/1955 [==============================] - 3846s 2s/step - loss: 0.1723 - categorical_accuracy: 0.9396 - val_loss: 0.1632 - val_categorical_accuracy: 0.9398
Epoch 1/1
1955/1955 [==============================] - 3852s 2s/step - loss: 0.1619 - categorical_accuracy: 0.9436 - val_loss: 0.1477 - val_categorical_accuracy: 0.9487
Epoch 1/1
1955/1955 [==============================] - 3835s 2s/step - loss: 0.1532 - categorical_accuracy: 0.9466 - val_loss: 0.1462 - val_categorical_accuracy: 0.9482
Epoch 1/1
1038/1955 [==============>...............] - ETA: 24:55 - loss: 0.1471 - categorical_accuracy: 0.9479

KeyboardInterrupt: ignored